In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import filter, col, first, round, concat, lit, when, to_date, trim
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

# conf = SparkConf()

# conf.setMaster("local").setAppName("My app")

# export SPARK_CLASSPATH=/path/to/downloaded/jar
# spark-submit --driver-class-path /path/to/downloaded/jar script.py

# bash:  export SPARK_CLASSPATH = C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar
# powershell: $env:SPARK_CLASSPATH = "C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar"
# $ pyspark
# spark-submit --jars "C:/Users/aiden2.kim/Desktop/Aiden-Projects/vacuum-fastapi-dash-etl-app/postgresql-42.6.0.jar" api/rotuers/crdi.py
# bin\pyspark --driver-class-path r"C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar" --jars r"C:\Users\aiden2.kim\Desktop\Aiden-Projects\vacuum-fastapi-dash-etl-app\postgresql-42.6.0.jar"


spark = SparkSession.builder \
    .appName('crdi-transform') \
    .config("spark.driver.extraClassPath", "C:/Users/aiden2.kim/Desktop/Aiden-Projects/vacuum-fastapi-dash-etl-app/postgresql-42.6.0.jar") \
    .getOrCreate()


# sdf = spark.read.csv('CRDI Appliances Test Insights Stick Vacuums Data.csv', header=True, inferSchema=True)
# sdf.printSchema()
# sdf.show()

In [46]:
# spark.stop()

In [2]:
df_input = spark.read.csv('CRDI Appliances Test Insights Stick Vacuums Data.csv', header=True, inferSchema=True)
df_input.printSchema()
# df_input.show()

root
 |-- category_name: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- rating_category_cro: string (nullable = true)
 |-- test_group: string (nullable = true)
 |-- test_measure: string (nullable = true)
 |-- run_id: string (nullable = true)
 |-- score_value: string (nullable = true)
 |-- units: string (nullable = true)
 |-- parent_brand_name: string (nullable = true)
 |-- parent_model_name: string (nullable = true)



In [3]:
ver_date = '2023-07-18'
# create a new column as category_id
df_input = df_input.withColumn('ver_date', lit(ver_date))\
                    .withColumn('test_category_id', lit(4))\
                    .withColumn('vac_type_id', when(col('category_name').like('%Cordless%'), 1).when(col('category_name').like('%Corded%'), 3))\
                    .withColumn('test_target_id', when(col('rating_category_cro').like('%Bare%'), 1).when(col('rating_category_cro').like('%Carpet%'), 2)\
                                .when(col('rating_category_cro').like('%Edges%'), 3).when(col('rating_category_cro').like('%Pet%'), 4)\
                                .when(col('rating_category_cro').like('%Noise%'), 5).when(col('rating_category_cro').like('%Convenience%'), 6)\
                                .when(col('rating_category_cro').like('%Spec%'), 7).when(col('rating_category_cro').like('%Clean%'), 8)\
                                .when(col('rating_category_cro').like('%score%'), 9).when(col('rating_category_cro').like('%Charge%'), 6) 
                               )\
                    .withColumn('test_group', when(col('test_measure') == 'Debris type', col('score_value')).otherwise(col('test_group')))\
                    .withColumn('test_group_id', when(col('test_group').like('%Sand%'), 1).when(col('test_group').like('%sand%'), 1)\
                                                .when(col('test_group').like('%Mix%'), 6).when(col('test_group').like('%Rice%'), 2)\
                                                .when(col('test_group').like('%Cheerio%'), 3).when(col('test_group').like('%Edge%'), 1)\
                                                .when(col('test_group').like('%Pet hair%'), 4).when(col('test_group').like('%Noise%'), 5)\
                                                .when(col('test_group').like('%Clearance%'), 7).when(col('test_group').like('%Run %'), 8)\
                                                .when(col('test_group').like('%Charge%'), 8).when(col('test_group').like('%Use force%'), 10)\
                                                .when(col('test_group').like('%Dust Bin%'), 6).when(col('test_group').like('%score%'), 16)\
                                                .when(col('test_group').like('%Clean emissions%'), 17).when(col('test_group').like('%Spec%'), 11)

                               
                               )
                    
                    
                               
            

df_input.show()
                                                                                                             
                                                                                                             
                                                                                                    
                    # .withColumn('test_group_id', col('another_column') + 10)\
                    # .withColumn('test_measure_id', col('another_column') + 10)\



+--------------------+----------+----------------+-------------------+---------------+--------------------+------+-----------+-----+-----------------+-----------------+----------+----------------+-----------+--------------+-------------+
|       category_name|brand_name|      model_name|rating_category_cro|     test_group|        test_measure|run_id|score_value|units|parent_brand_name|parent_model_name|  ver_date|test_category_id|vac_type_id|test_target_id|test_group_id|
+--------------------+----------+----------------+-------------------+---------------+--------------------+------+-----------+-----+-----------------+-----------------+----------+----------------+-----------+--------------+-------------+
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1B|        Bare floors|           Rice|         Debris type|     1|       Rice| null|             null|             null|2023-07-18|               4|          1|             1|            2|
|Stick Vacuums (Co...|    Makita|18V LXT XLC02R1

In [4]:
df_input.createOrReplaceTempView("input")

In [5]:
spark.sql("""
    SELECT DISTINCT
        test_target_id,
        rating_category_cro,
        test_group_id,
        test_group,
        test_measure,
        score_value,
        run_id
    FROM input
    WHERE test_target_id = 1
   

    """).show(1000)


+--------------+-------------------+-------------+-----------+--------------------+-----------+------+
|test_target_id|rating_category_cro|test_group_id| test_group|        test_measure|score_value|run_id|
+--------------+-------------------+-------------+-----------+--------------------+-----------+------+
|             1|        Bare floors|         null|Bare floors|   Pickup Percentage|         81|     6|
|             1|        Bare floors|         null|Bare floors| Final Vacuum Weight|     3489.2|     5|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     3527.5|     3|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     2518.9|     3|
|             1|        Bare floors|         null|Bare floors|Initial Vacuum We...|     3390.4|     2|
|             1|        Bare floors|         null|Bare floors|   Pickup Percentage|         97|     1|
|             1|        Bare floors|         null|Bare floors|   Pickup P

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import first

# Define a window specification for bare
window_spec = Window.partitionBy('model_name', 'test_target_id', 'run_id')

# Create a new DataFrame with the updated 'test_group_id' column for test_target_id = 1
df_bare = df_input.filter((col("test_target_id") == 1) | (col("test_target_id") == 2))

# drop df_input(test_target_id = 1) from df_input
df_input = df_input.filter((col("test_target_id") != 1) & (col("test_target_id") != 2))
rows = df_input.count()
print(rows)

df_bare = df_bare.withColumn('test_group', first('test_group').over(window_spec))\
                  .withColumn('test_group_id', first('test_group_id').over(window_spec))

# append df_bare to df_input
df = df_input.union(df_bare)



7000


In [7]:
df = df.withColumn('test_group_id', when((col('test_target_id') == 6) & (col('test_group_id').isNull()), 16).otherwise(col('test_group_id')))

In [8]:
df.createOrReplaceTempView("input")

In [9]:
spark.sql("""
    SELECT DISTINCT
        test_target_id,
        rating_category_cro,
        test_group_id,
        test_group,
        test_measure,
        score_value,
        run_id
    FROM input
  

    """).show(1000)



+--------------+-------------------+-------------+--------------------+--------------------+------------+------+
|test_target_id|rating_category_cro|test_group_id|          test_group|        test_measure| score_value|run_id|
+--------------+-------------------+-------------+--------------------+--------------------+------------+------+
|             3|              Edges|            1|               Edges|Front Edge Distance4|         0.5|     1|
|             3|              Edges|            1|               Edges| Left Edge Distance1|        0.25|     1|
|             6|        Convenience|           10|           Use force|        Avg Vac Push| -3.72871245|     1|
|             8|    Clean emissions|           17|     Clean emissions| Vacuum Accumulation|       25.89|     1|
|             6|        Convenience|            7|           Clearance|           Distance8|       31.75|     1|
|             7|               Spec|           11|                Spec|Can Stand On Its Own|    

In [207]:
import openpyxl
# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = df.toPandas()

# Save the Pandas DataFrame as an Excel file
pandas_df.to_excel("output3.xlsx", index=False)

In [30]:
spark.sql("""
    SELECT DISTINCT
        test_group,
        test_measure,
        score_value
    FROM input
    WHERE score_value IS NOT NULL
        AND TRY_CAST(score_value AS DOUBLE) IS NULL
        AND TRY_CAST(test_group AS DOUBLE) IS NULL

    """).show(1000)


+------------------+--------------------+------------------+
|        test_group|        test_measure|       score_value|
+------------------+--------------------+------------------+
|       Charge time|         Charge time|      Corded model|
|    Run Time Score|Manufacturer Powe...|        Boost Mode|
|              Spec|        Brush on/off|               Yes|
|              Spec|See-through Dirt Bin|               Yes|
|         Use force|Manufacturer Powe...|               Eco|
|         Clearance|Manufacturer Powe...|          Standard|
|         Use force|Intended Power Level|               Low|
|         Use force|Intended Power Level|              High|
|              Spec|Battery Status In...|               Yes|
|    Run Time Score|Manufacturer Powe...|            medium|
|              Spec|Indicator Light f...|               Yes|
|    Run Time Score|Manufacturer Powe...|          Standard|
|    Run Time Score|Manufacturer Powe...|             boost|
|    Run Time Score|Manu

In [ ]:
df = df.withColumn('test_measure_id', when(col('test_measure') == 'Debris Weight', 0)\
                    .when(col('test_measure').like('%Initial%Weight'), 1).when(col('test_measure').like('Final%Weight'), 2)\
                    .when(col('test_measure').like('%Mix%'), 6).when(col('test_group').like('%Rice%'), 2)\
                    .when(col('test_group').like('%Cheerio%'), 3).when(col('test_group').like('%Edge%'), 1)\
                    .when(col('test_group').like('%Pet hair%'), 4).when(col('test_group').like('%Noise%'), 5)\
                    .when(col('test_group').like('%Clearance%'), 7).when(col('test_group').like('%Run %'), 8)\
                    .when(col('test_group').like('%Charge%'), 8).when(col('test_group').like('%Use force%'), 10)\
                    .when(col('test_group').like('%Dust Bin%'), 6).when(col('test_group').like('%score%'), 16)\
                    .when(col('test_group').like('%Clean emissions%'), 17).when(col('test_group').like('%Spec%'), 11)


                               )


In [ ]:
'''
test_measure_id:
{
    0 - Soil Weight
    1 - Weight - Before
    2 - Weight - After
    3 -	Weight - Diff.
    4 - Pickup
    5 - Rug Weight - Before
    6 - Rug Weight - After
    7 -	Rug Weight - Diff.
    8 -	Relative Humidity
    9 - Room  Temperature
    10 - Edges Front 1
    11 - Edges Front 2
    12 - Edges Front 3
    13 - Edges Front 4
    14 - Edges Front 5
    15 - Edges Front 6
    16 - Edges Front Avg.
    17 - Edges Left 1
    18 - Edges Left 2
    19 - Edges Left 3
    20 - Edges Left Avg.
    21 - Edges Right 1
    22 - Edges Right 2
    23 - Edges Right 3
    24 - Edges Right Avg.
    25 - Edges Pickup Left
    26 - Edges Pickup Right
    27 - Removal Strokes
    28 - Clearance
    29 - Clearance 3
    30 - Clearance 4
    31 - Clearance 5
    32 - Clearance 6
    33 - Clearance 7
    34 - Clearance 8
    35 - Clearance Avg.
    36 - Score
    37 - Capacity
    38 - Ease
    39 - Empty Weight
    40 - Full Weight
    41 - Features
    42 - Power
    43 - Charge
    44 - Runtime
    45 - Minutes
    46 - Power Level
    47 - Power Supply
    48 - Power Brush Off
    49 - Seconds
    50 - Pull - Avg. Lb
    51 - Push - Avg. Lb
    52 - Pull - Avg. Vac
    53 - Push - Avg. Vac
    54 - Pull - Avg. Vac Not Attached
    55 - Push - Avg. Vac Not Attached
    56 - Corldess
    57 - Mop
    58 - Weight
    59 - Cord Length
    60 - Self-Stand
    61 - Battery Status Indicator - Percentage
    62 - Brush On/Off
    63 - Auto-Suction Mode
    64 - Power Mode Indicator
    65 - Headlight
    66 - Easy-to-Maneuver
    67 - Head Swivels
    68 - Anti Hair-tangle
    69 - Dust Sensor

}


'''

In [ ]:
# # sdf to sql table
# sdf.createOrReplaceTempView('crdi')

'''
# test_details schema

id	                        int (PK)
test_parent_id	            int (FK to tests.id)
test_category_id	        int (FK to test_categories.id)
vac_type_id	                int (FK to vac_types.id)
test_target_id	            int (FK to test_targets.id)
test_group_id               int (FK to test_groups.id)
test_measure_id	            int (FK to test_measures.id)
test_case                   str
tester                      int (FK to users.id)
inv_no                      int (FK to vacuums.id)
brush_type                  str
test_measure                str
value                       str
units                       str
run                         int
run_date                    timestamp
notes                       str
image                       str
owner_id                    int (FK to users.id)

'''

'''
for CRDI data:

test_category_id = 4
vac_type_id: {'Stick Vacuums (Cordless)': 1, 'Stick Vacuums (Corded)': 3}
test_target_id: rating_category_cro

{
     1 - Bare Floors
     2 - Carpet
     3 - Edges
     4 - Pet hair
     5 - Noise
     6 - Convenience
     7 - Specs
     8 - Clean Emissions
     9 - Overall Score,

}

test_group_id:

{
    1 - Sand
    2 - Rice
    3 - Cheerios
    4 - Strokes
    5 - Clearance
    6 - Dustbin
    7 - Features
    8 - Power
    9 - Usability
    10 - Runtime
    11 - Use-Force
    12 - BK-2230
    13 - Charge Time
    14 - Spec
    15 - Bag
    16 - Vacuum Body
    17 - Score
    18 - Pull Ease
    19 - Push Ease

}


'''




In [ ]:
# connect to db

df_vacuum = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "vacuums") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_vacuum.printSchema()


In [ ]:
df_test = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "tests") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_test.printSchema()

In [ ]:
df_test_vacuums = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "test_vacuums") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_test_vacuums.printSchema()

In [ ]:
df_crdi = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vacuum_api") \
    .option("dbtable", "crdi") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_crdi.printSchema()

In [ ]:
# register temp tables

df_vacuum.createOrReplaceTempView("vacuum")
df_test.createOrReplaceTempView("test")
df_test_vacuums.createOrReplaceTempView("test_vacuums")
df_crdi.createOrReplaceTempView("crdi")



In [ ]:
spark.sql("""
    SELECT *
    FROM input
    LIMIT 10
    """).show()


In [ ]:
df_input = spark.sql("""
    SELECT * FROM input
    """)

In [ ]:
df_input.show()